In [ ]:
import cv2
import numpy as np
from timeit import default_timer as timer
import datetime
import math
import os
import shutil
dirpath = "screenshots"
for filename in os.listdir(dirpath):
    filepath = os.path.join(dirpath, filename)
    try:
        shutil.rmtree(filepath)
    except OSError:
        os.remove(filepath)

# data
cwd = os.getcwd()
examples_dir = cwd + "\\video\\"
bad_video = examples_dir + "bad.wmv"
good_video = examples_dir + "good.wmv"

# settings
turn_on_delay = 5
x,y = 200,190
w,h = 1070,150
frames_limit = 10
detection_zones = 6
# bound_velocity = 0.006
bound_velocity = 0.006

mycounter = 0
print(mycounter)

class ImageFeature:
    def __init__(self, area_ratio):
        self.area_ratio=area_ratio
        
class SegmentAnswer:
    def __init__(self, feature, was_detected):
        self.feature=feature
        self.was_detected=was_detected

class FrameBuffer:
    def __init__(self):
        self.features_array = [ [ ImageFeature(0.0) for i in range(detection_zones) ] for j in range(frames_limit) ]

frames_buffer = FrameBuffer()        
        
# Common memory
measures = [ [ 0 for i in range(detection_zones) ] for j in range(frames_limit) ]
errorDelayCounters = [0]*detection_zones


def printLabel( image, label, upper ):
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # org 
    org = ((50, 50),(50,100))[upper]
    # fontScale 
    fontScale = 1
    # Blue color in BGR 
    color = (255, 0, 0) 
  
    # Line thickness of 2 px 
    thickness = 2
    return cv2.putText(image, label, org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)

def extract_features(image):
    fullArea = image.shape[0]*image.shape[1]
    area = cv2.countNonZero(image)
    area_ratio = area/fullArea
    return ImageFeature(area_ratio)
        
def process_zone( zone_index, image):
    array=frames_buffer.features_array[zone_index]
    image_feature = extract_features(image)
    
    feature = ImageFeature(0.0)
    fullArea = image.shape[0]*image.shape[1]
    area = cv2.countNonZero(image)
    ratio = area/fullArea
    if( len(array) > frames_limit):
        frames_buffer.features_array[zone_index].pop(0)
        return  SegmentAnswer(0.0,False)
    frames_buffer.features_array[zone_index].append(ImageFeature(ratio))
    # Check the matrix to fill last element with spread
    array=frames_buffer.features_array[zone_index]
    ratio_array = []
    for items in array:
        ratio_array.append(items.area_ratio)
    f_min = min(ratio_array)
    f_max = max(ratio_array)
    diffs = []
    for index in range(0,len(ratio_array)-1):
        diffs.append(math.fabs(ratio_array[index]-ratio_array[index+1]))
    if( max(diffs) > bound_velocity ):
        return SegmentAnswer(max(diffs),True)
    return SegmentAnswer(max(diffs),False)
            

# возвращает [(порог превышен, максимальная производная метрики)]
def detect_artifacts(binarized_image,zones_count,x,y,w,h):
    answer = []
    for zone_index in range(0,zones_count):
        zone_width = int(w/zones_count)
        rect_x = x + zone_index*zone_width
        cropped_zone = binarized_image[y+1:y+h-2,rect_x:rect_x+zone_width-2]
        feature = process_zone(zone_index, cropped_zone)
        answer.append(feature)
    return answer

def addZoneRect(color_image, zone_index, zones_count, green_or_red, x,y,w,h ):
    zone_width = int(w/zones_count)
    rect_x = x + zone_index*zone_width
    color = ((1,255,1),(1,1,255))[green_or_red]
    dept=(4,1)[green_or_red]
    return cv2.rectangle(color_image,(rect_x,y),(rect_x+zone_width,y+h),color,dept)
    
activation_answers = [0 for i in range(detection_zones)]    
counter = 0

def process_frame(frame, images, is_ready):
    gr_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret,binarizedImage = cv2.threshold(gr_img,215,255,cv2.THRESH_TOZERO)
    color_img = cv2.cvtColor(binarizedImage, cv2.COLOR_GRAY2RGB)
        
    images.append(gr_img)
    if( len(images)>3 ):
        images.pop(0)

    if( len(images)==3):      
        # im_t is the frame of interest; im_tp1 and im_tm1 are, respectively
        # the successive and previous frames.
        dbp = cv2.absdiff(images[1], images[0])
        db0 = cv2.absdiff(images[2], images[0])
        dbm = cv2.absdiff(images[1],images[2])
        detect = cv2.bitwise_not( cv2.bitwise_and(cv2.bitwise_and(dbp, dbm),cv2.bitwise_not(db0)))
        r,bb = cv2.threshold(detect,215,255,cv2.THRESH_TOZERO_INV)
        thresh_frame = bb #cv2.dilate(bb, None, iterations = 2) 
        cv2.imshow('Original44',bb)
        cv2.imshow('Original46',thresh_frame)
        binarizedImage = thresh_frame

    label = ""
    label_activation = ""
    if is_ready:
        answer_metrics_pairs = detect_artifacts( binarizedImage, detection_zones,x,y,w,h)
        for zone_index in range(detection_zones):
            (answer,metrics)=(answer_metrics_pairs[zone_index].was_detected, answer_metrics_pairs[zone_index].feature)
            label = label +("Y","N")[answer]+" "+"{:.5f}".format(metrics)+" "
            longError =  errorDelayCounters[zone_index] > 0
            if longError:
                answer = True
                errorDelayCounters[zone_index] =  errorDelayCounters[zone_index] - 1
            elif answer:
                errorDelayCounters[zone_index] = 4
                global mycounter
                activation_answers[zone_index]= metrics
                cv2.imwrite("screenshots/frame%d.jpg" % mycounter, binarizedImage)     # save frame as JPEG file
                mycounter = mycounter + 1
            color_img = addZoneRect( color_img, zone_index, detection_zones, answer, x,y,w,h)
    else:
        label = "focusing delay"

    for activation in activation_answers:
        label_activation = label_activation + "{:.5f}".format(activation) + " "
    finalImage = printLabel(color_img, label,True)
    finalImage = printLabel(finalImage, label_activation,False)
    cv2.imshow('Original',frame)
    cv2.imshow('Frame',finalImage)

def displayProgressBar(capture, currentframe):
    #get next frame number out of all the frames for video
    nextFrameNo = capture.get(cv2.CAP_PROP_POS_FRAMES)
    #get total number of frames in the video
    totalFrames =capture.get(cv2.CAP_PROP_FRAME_COUNT)
    #calculate the percent complete based on the frame currently
    #playing. OpenCV does provide a variable to access this
    #property directly (CAP_PROP_POS_AVI_RATIO), however
    #it seems to not work all the time, hence we calculate internally
    complete = nextFrameNo/totalFrames
    return complete

def update_print_progress(last_progress, frame, capture):
    multiplier = 10000
    progress = multiplier*displayProgressBar(capture,frame)
    if int(progress)-int(last_progress) >= 1:
        print("{:.5f}".format(progress/multiplier))
        return progress
    return last_progress

def process_video_file(video_path):
    # Focusing delay
    start = timer()
        
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return True, "Error opening video stream or file"
    
    images = []
    progress = -1
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        # progress = update_print_progress(progress, frame, cap)   
        is_ready =  timer() - start > turn_on_delay    
        process_frame(frame, images, is_ready)
        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
       

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    return False,""

process_video_file(bad_video)
print('final')
print(activation_answers)

0


In [ ]:
from IPython.display import clear_output

import random
import socket
import time

port = 9090
message_max_length = 1024
connectionsCount = 0
screenFlipper = False
msg = '!{0}"{1}"{2}"{3}"{4}"{5}'

def CreateRandomMessage():
    global screenFlipper
    global msg
    msg_copy = msg
    errorFound = random.randint(0,1)
    isWorking = random.randint(0,1)
    flipping = (0,1)[screenFlipper]
    screenFlipper = not screenFlipper
    possibility=str(random.randint(0,100)).zfill(3)
    size = str(random.randint(0,999)).zfill(3)
    msg_copy = msg_copy.format(errorFound,isWorking,flipping,errorFound,possibility,size)
    return msg_copy.encode()

def SocketCycle(conn):
    global message_max_length
    while True:
        data = conn.recv(message_max_length)
        if not data:
            break
        conn.send(CreateRandomMessage())

def SocketHandle(sock):
    print("Server started")
    global port
    sock.bind(('', port))
    sock.listen(1)
    conn, addr = sock.accept()
    print('connected:', addr)
    SocketCycle(conn)

    
def Main():
    global connectionsCount
    while True:
        sock = socket.socket()
        SocketHandle(sock)
        sock.close()
        clear_output(wait=True)
        print("Server stopped due to disconnect, restarting")
        print("Restart count = {0}".format(connectionsCount))
        connectionsCount = connectionsCount + 1
    
    
Main()
   


Server stopped due to disconnect, restarting
Restart count = 0
Server started
